In [2]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.utils import AnalysisException
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql.functions import udf, mean
from pyspark.sql.types import StringType, StructType, StructField, FloatType
import pandas as pd
import numpy as np
import os, math, time
import itertools
import csv

In [3]:
t0 = time.time()

In [4]:
conf = (SparkConf()
        #.setMaster('spark://10.100.5.182:7077')
        #.setMaster("local[4]")
        #.set("spark.executor.memory", "6g")
        .setAppName("quiz"))

In [5]:
try:
    sc = SparkContext(conf=conf)
    sql_sc = SQLContext(sc)
except ValueError:
    pass

In [6]:
def read_csv(file_name):
    try:
        data = sql_sc.read.csv(file_name, 
                       sep=',', 
                       header=False, 
                       mode='DROPMALFORMED')
    except AnalysisException:
        data = sql_sc.read.csv('hdfs:///bdm/quiz/{}'.format(file_name), 
                       sep=',', 
                       header=False, 
                       mode='DROPMALFORMED')
    return data

In [7]:
data = read_csv('kddcup.data')

In [8]:
cloumnnames = ["duration",
               "protocol_type", 
               "service",
               "flag",
               "src_bytes",
               "dst_bytes",
               "land",
               "wrong_fragment",
               "urgent",
               "hot",
               "num_failed_logins",
               "logged_in",
               "num_compromised",
               "root_shell",
               "su_attempted",
               "num_root",
               "num_file_creations",
               "num_shells",
               "num_access_files",
               "num_outbound_cmds",
               "is_host_login",
               "is_guest_login",
               "count",
               "srv_count",
               "serror_rate",
               "srv_serror_rate",
               "rerror_rate",
               "srv_rerror_rate",
               "same_srv_rate",
               "diff_srv_rate",
               "srv_diff_host_rate",
               "dst_host_count",
               "dst_host_srv_count",
               "dst_host_same_srv_rate",
               "dst_host_diff_srv_rate",
               "dst_host_same_src_port_rate",
               "dst_host_srv_diff_host_rate",
               "dst_host_serror_rate",
               "dst_host_srv_serror_rate",
               "dst_host_rerror_rate",
               "dst_host_srv_rerror_rate",
               "intrusion_type"]

In [9]:
for old, new in zip(['_c%d'%(i) for i in range(0, 42)], cloumnnames):
    data = (data.withColumnRenamed(old, new))
data = (data.withColumn('duration', data.duration.cast('float'))
       .withColumn('src_bytes', data.src_bytes.cast('float'))
       .withColumn('dst_bytes', data.dst_bytes.cast('float'))
       .withColumn('num_failed_logins', data.num_failed_logins.cast('float')))

In [10]:
data = data.dropna()

In [11]:
data = data.sample(False, 0.01, 42)

### (1) For continuous attributes ‘duration’, ‘src_bytes’, ‘dst_bytes’, ‘num_failed_logins’, please calculate their mean, median, mode, standard deviation, respectively

In [62]:
def find_median(values_list):
    try:
        median = np.median(values_list) #get the median of values in a list in each row
        return round(float(median),2)
    except Exception:
        return None #if there is anything wrong with the given values

median_finder = udf(find_median, FloatType())

In [63]:
q1_feature_nemaes = ['duration', 'src_bytes', 'dst_bytes', 'num_failed_logins']

In [64]:
q1 = data.select('duration', 'src_bytes', 'dst_bytes', 'num_failed_logins')

In [68]:
print(q1.describe().show())

+-------+------------------+-----------------+-----------------+--------------------+
|summary|          duration|        src_bytes|        dst_bytes|   num_failed_logins|
+-------+------------------+-----------------+-----------------+--------------------+
|  count|             49325|            49325|            49325|               49325|
|   mean|48.422807906741006|792.3461530663964| 686.317952356817|1.013684744044602...|
| stddev| 701.3569789920128|23451.35807800003|14356.54144803163| 0.01191253397088856|
|    min|               0.0|              0.0|              0.0|                 0.0|
|    max|           42221.0|        5133876.0|        1649990.0|                 2.0|
+-------+------------------+-----------------+-----------------+--------------------+

None


In [ ]:
for f in q1_feature_nemaes:
    print('%s median:%f'%(f, q1.approxQuantile(f, [0.5], 0.25)[0]))

In [ ]:
for f in q1_feature_nemaes:
    print('%s mode is'%(f), q1.select(f).rdd
    .map(lambda x: (str(x[0]), 1))
    .reduceByKey(lambda a,b:a+b)
    .sortBy(lambda w: w[1], ascending=False)
    .take(1))

### (2) For symbolic attributes ‘protocol_type’, ‘service’, ‘flag’, ‘logged_in’, ‘intrusion_type’, output the list of each value and the corresponding frequency count, sorted in descending order of the count


In [66]:
q2_feature_names = ['protocol_type', 'service', 'flag', 'logged_in', 'intrusion_type']

In [69]:
# (data.select('protocol_type').rdd
# .map(lambda x: (x[0], 1))
# .reduceByKey(lambda a,b:a+b)
# .sortBy(lambda w: w[1], ascending=False)
# .collect())

[('icmp', 28422), ('tcp', 18964), ('udp', 1939)]

In [72]:
for f in q2_feature_names:
    r = (data.select(f).rdd
        .map(lambda x: (x[0], 1))
        .reduceByKey(lambda a,b:a+b)
        .sortBy(lambda w: w[1], ascending=False)
        .collect())
    print('[Q2] column:%s value and the corresponding frequency count'%(f), r)

q2 column:protocol_type value and the corresponding frequency count [('icmp', 28422), ('tcp', 18964), ('udp', 1939)]
q2 column:service value and the corresponding frequency count [('ecr_i', 28187), ('private', 11022), ('http', 6350), ('smtp', 1027), ('other', 781), ('domain_u', 574), ('ftp_data', 390), ('eco_i', 171), ('finger', 69), ('urp_i', 61), ('ftp', 55), ('telnet', 45), ('auth', 40), ('ntp_u', 35), ('pop_3', 22), ('ssh', 19), ('time', 19), ('vmnet', 17), ('ldap', 16), ('exec', 16), ('domain', 15), ('sql_net', 15), ('Z39_50', 14), ('efs', 14), ('netbios_ssn', 14), ('supdup', 13), ('netbios_dgm', 13), ('shell', 13), ('echo', 13), ('bgp', 12), ('mtp', 12), ('nntp', 12), ('uucp', 12), ('remote_job', 11), ('whois', 11), ('http_443', 11), ('discard', 11), ('login', 10), ('netstat', 10), ('rje', 10), ('hostnames', 10), ('gopher', 10), ('courier', 10), ('netbios_ns', 10), ('pop_2', 10), ('systat', 10), ('csnet_ns', 9), ('printer', 9), ('ctf', 8), ('daytime', 8), ('name', 8), ('sunrpc', 

### (3) Output the list of the most frequently used ‘service’ for each ‘intrusion_type’, sorted in descending order of the occurrence frequency

In [83]:
q3_result =  (data.select('service', 'intrusion_type').rdd
.map(lambda x: ((x[0], x[1]), 1))
.reduceByKey(lambda a,b: a+b)
.map(lambda x: (x[0][1], (x[0][0], x[1])))
.groupByKey()
.map(lambda x: (x[0], sorted(x[1], key=lambda x: x[1], reverse=True)[0]))
.collect())
for intrusion_type, r in q3_result:
    print('[Q3]intrusion_type:[%15s],  most frequently used service is:%20s'%(intrusion_type, str(r)))

intrusion_type:[   warezclient.],  most frequently used service is:     ('ftp_data', 4)
intrusion_type:[      teardrop.],  most frequently used service is:     ('private', 14)
intrusion_type:[         satan.],  most frequently used service is:      ('other', 143)
intrusion_type:[         smurf.],  most frequently used service is:    ('ecr_i', 28154)
intrusion_type:[          back.],  most frequently used service is:        ('http', 15)
intrusion_type:[       neptune.],  most frequently used service is:  ('private', 10188)
intrusion_type:[    loadmodule.],  most frequently used service is:       ('telnet', 1)
intrusion_type:[       ipsweep.],  most frequently used service is:      ('eco_i', 129)
intrusion_type:[      multihop.],  most frequently used service is:          ('ftp', 1)
intrusion_type:[     portsweep.],  most frequently used service is:     ('private', 58)
intrusion_type:[  guess_passwd.],  most frequently used service is:       ('telnet', 2)
intrusion_type:[           pod.]

### (4) If we regard the values of ‘intrusion_type’ except “normal” as abnormal, calculate the correlation coefficient of “number of abnormal instances” and ‘num_failed_logins’ by the following formula:


In [12]:
q4 = data.select('intrusion_type', 'num_failed_logins')

In [13]:
q4 = (q4.withColumn('is_normal', q4.intrusion_type == 'normal.'))
q4 = q4.withColumn('is_normal', q4.is_normal.cast('int'))

In [14]:
corr_coe = q4.select('is_normal', 'num_failed_logins').collect()

In [15]:
len(corr_coe)

49325

In [ ]:
corr_coe = np.corrcoef(np.array(corr_coe))

In [ ]:
sc.stop()

In [ ]:
print('cost {:.3f} minutes'.format((time.time()-t0)/60))